# README

---

VCP SDKを用いてクラウド上にOpenHPC環境を構築します。

## 概要

[OpenHPC](https://openhpc.community/)で配布しているパッケージを利用してクラウド上にHPC環境を構築します。

### 構成について

ここで構築する環境は１つのマスターノード（ヘッドノード）と複数の計算ノードによって構成されます。
マスターノードはNFSサーバとしての役割も担います。NFSによってホームディレクトリやOpenHPCのファイルを計算ノードとの間で共有します。

OpenHPCではジョブスケジューラとして[Slurm](https://www.schedmd.com/)と[PBS Professional](http://www.pbspro.org/)が選択できますが、ここではSlurmを使用します。

![構成](images/ohpc-000.png)

### 構築方法について

OpenHPCのInstall guideでは[Warewulf](http://warewulf.lbl.gov/)などを用いて計算ノードのプロビジョニングを行っていますが、ここでは Warewulf などのソフトウェアを利用しません。マスターノードや計算ノードにはVCP SDKを用いてクラウド上に作成した仮想サーバを利用します。同様にNFS用の仮想ディスクもVCP SDKによって作成します。VCP SDKが作成する仮想サーバ、仮想ディスクのことを、ここではVCノード、VCディスクと呼びます。

#### VCノードの作成

VCノード（クラウド上の仮想サーバ）を作成するには、このNotebook環境に用意されているVCP SDKを利用します。VCP SDKはNotebook環境から利用できるPython3のライブラリで、パブリッククラウド上(AWS, Azure, ...)に仮想サーバや仮想ディスクを作成するなどの操作が行えます。

実際のクラウドに対する操作はVCP SDKが直接行うわけではなくVC Controllerと呼ばれるVCPのサーバが実行しています。VC Controllerの機能をNotebook環境から直接利用することも可能ですが、利用しやすいようにまとめたライブラリが VCP SDKになります。VC Contorollerを利用するには、アクセストークンが必要となります。同様にVCP SDKを利用する場合も、VC Controllerのアクセストークンが必要となります。VCP SDKによる操作を行うために、事前にVC管理者に対してアクセストークンの発行を依頼しておいてください。

#### VCノードのイメージ選択

VCPでは利用者に対して、異なるクラウド(AWS, Azure, ...)の仮想サーバを同一の環境として提供するために[Docker コンテナ](https://www.docker.com/)を利用しています。クラウド上のVM/BMをVCノードとして組み入れる際にVCPが各VM/BMの上で共通環境となるコンテナを起動します。ここでは、このコンテナのことをBaseコンテナと呼びます。Baseコンテナは仮想サーバのモニタリングなどのVCノードに共通する機能を提供するための役割も担っています。



標準のBaseコンテナは共通環境を提供することを目的としていますが、それとは別に特定の利用目的を持ったBaseコンテナを
アプリケーションテンプレートでは提供しています。OpenHPC向けには、マスターノード用Baseコンテナイメージと計算ノード用Baseコンテナイメージを用意しています。ここでは、それらのコンテナイメージを利用します。

#### VCディスクの作成

VCディスク（クラウド上の仮想ディスク）もVCノードと同様にVCP SDKを利用することで作成できます。

現在VCディスクを作成できるプロバイダは `AWS` と `Azure` に限定されています。その他のプロバイダでOpenHPC環境を構築することを考慮して、このアプリケーションテンプレートではマスターノードにNFS用ディスクを作成しない構成にも対応しています。NFS用ディスクを作成しない場合は仮想ノードのルートボリュームを直接利用するので、ルートボリュームサイズに大きめの値を指定してください。

### バージョン

このNotebookが構築するミドルウェア、OSのバージョンを以下に示します。

* OpenHPC 1.3.9
* CentOS 7.7

## Notebookの一覧

お手本Notebookの一覧を示します。

**注意**:

この節ではお手本Notebookへのリンクを示す箇所がありますが、リンク先のNotebookは参照用となっていて**そのままでは実行できません**。

> Notebook自体は実行できてしまいますが、パスなどが想定しているものと異なるため正しく処理できずエラーとなります。

次のどちらかの手順で作業用Notebookを作成する必要があります。

* 次節の「作業用Notebookの作成」で作業用のNotebookを作成する。
* お手本Notebookを配置してある `notebooks/` から、この`000-README.ipynb`と同じディレクトリにNotebookをコピーする。

### 各Notebookの関連について

次のセルを実行すると、各お手本Notebookの関連を示す図を表示します。

> 図が表示されずに `<IPython.core.display.SVG object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

図に表示される１つのブロックが１つのNotebookに対応しており、ブロックのタイトル部分にお手本Notebookへのリンクが埋め込まれています。

In [ ]:
from IPython.display import SVG
%run scripts/nb_utils.py
setup_diag()
SVG(filename=generate_svg_diag(diag='images/notebooks.diag'))

### 各Notebookの目次

次のセルを実行すると、各お手本Notebookの目次が表示されます。

> 目次が表示されずに `<IPython.core.display.Markdown object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

リンクが表示されている項目が一つのNotebookに対応しており、そのサブ項目が各Notebook内の目次になっています。

In [ ]:
from IPython.display import Markdown
%run scripts/nb_utils.py
Markdown(notebooks_toc())

## 作業用Notebookの作成

この節のセルを実行することで、お手本Notebookから作業用Notebookを作成することができます。

まず、作業用Notebookを配置するディレクトリを指定してください。

In [ ]:
WORK_DIR = 'work'

次のセルを実行すると、Notebook名のドロップダウンリストと「作業開始」ボタンが現れます。
「作業開始」ボタンを押すと、お手本Notebookのコピーを作成した後、自動的にブラウザでコピーが開きます。
Notebookの説明を確認しながら実行、適宜修正しながら実行していってください。

> このNotebookを Shutdown した後に再度開いた場合、次のセルに既に表示されている「作用開始」ボタンが正しく動作しません。次のセルをいったん unfreeze した後、セルを再実行してから「作業開始」ボタンをクリックして下さい。

In [ ]:
from IPython.core.display import HTML
%run scripts/nb_utils.py
setup_nb_workdir(WORK_DIR)
HTML(generate_html_work_nbs(WORK_DIR))